In [1]:
import json

with open('GSPC-Yahoo-Finance.json') as f:
	data = json.load(f)

import pandas as pd

df = pd.DataFrame()

df['timestamp'] = data['chart']['result'][0]['timestamp']

quotes = data['chart']['result'][0]['indicators']['quote'][0]

for quote in quotes.keys():
	df[quote] = quotes[quote]

df['adjclose'] = data['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

# Calculate bear markets

inBearMkt = False
lastPeak = None
bearMarketThreshold = 0.2 # Decline since peak when the market is officially bearish

bearMarkets = []

for i, day in df.iterrows():
	if lastPeak == None:
		lastPeak = day.to_dict()

	if not inBearMkt:
		if day['close'] > lastPeak['close']:
			lastPeak = day.to_dict()
		if day['close'] <= lastPeak['close'] * (1-bearMarketThreshold):
			inBearMkt = True
			bearMarkets.append({'start': lastPeak['timestamp'], 'trigger': day['timestamp']})
	
	if inBearMkt:
		if day['close'] >= lastPeak['close']:
			bearMarkets[-1]['end'] = day['timestamp']
			inBearMkt = False
	
# If we are currently in a bear market, add an end to the current bear market
if 'end' not in bearMarkets[-1].keys():
	bearMarkets[-1]['end'] = df.iloc[-1]['timestamp']

import plotly.graph_objects as go

from datetime import date, datetime
from dateutil import relativedelta

def YMDBetweenDates(date2, date1):
	diff = relativedelta.relativedelta(date2, date1)
	output = ''
	ymd = {}
	ymd['yrs'] = diff.years
	ymd['mos'] = diff.months
	ymd['days'] = diff.days
	for x in ymd.items():
		if x[1] < 1:
			continue
		if x[1] < 2:
			output += str(x[1]) + ' ' + str(x[0])[:-1] + ' '
		else:
			output += str(x[1]) + ' ' + str(x[0]) + ' '
	return output


bearData = []

for bear in bearMarkets:
	mask = (df['timestamp'] >= bear['start']) & (df['timestamp'] <= bear['end'])
	bearData.append(df.loc[mask])

for data in bearData:
	data['days'] = range(len(data))
	pos = data.columns.get_loc('close')
	data['change'] =  (data.iloc[1:, pos] / data.iat[0, pos]) -1



currentMarket = df[df['timestamp'] > pd.Timestamp(2022, 1, 2)]
currentMarket['days'] = range(len(currentMarket))
pos = currentMarket.columns.get_loc('close')
currentMarket['change'] =  (currentMarket.iloc[1:, pos] / currentMarket.iat[0, pos]) - 1

C:\Users\ash\AppData\Local\Temp/ipykernel_12416/1490141476.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['days'] = range(len(data))
C:\Users\ash\AppData\Local\Temp/ipykernel_12416/1490141476.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['change'] =  (data.iloc[1:, pos] / data.iat[0, pos]) -1
C:\Users\ash\AppData\Local\Temp/ipykernel_12416/1490141476.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [20]:
for i, data in reversed(list(enumerate(bearData[:-1]))):
	name = data.iloc[0]['timestamp'].strftime("%b-%Y")
	if name != 'Oct-2007':
		continue
	fig = go.Figure()
	fig.add_trace(go.Scatter(x=currentMarket['days'], y=currentMarket['change'], mode='lines', name='Current Market', line={'width': 3, 'color': 'darkblue'}, 
		hovertemplate=[f" {x['change']:.1%} ({x['timestamp'].strftime('%b %d, %Y')})" for i, x in currentMarket.iterrows()]
		))
	fig.add_trace(go.Scatter(x=bearData[i]['days'], y=bearData[i]['change'], mode='lines', name=name,
		hovertemplate=[f" {x['change']:.1%} ({x['timestamp'].strftime('%b %d, %Y')})" for i, x in bearData[i].iterrows()]
		))

	minCandle = bearData[i][bearData[i].close == bearData[i].close.min()]

	# Peak
	fig.add_annotation(x=0, 
		y=0,
		text=f'''┌ Peak''',
		align='left',
		showarrow=False,
		yshift=9,
		xshift=-5,
		xanchor='left'
		)

	# Market bottom
	fig.add_annotation(x=minCandle['days'].values[0], 
		y=minCandle['change'].values[0],
		text=f'''└ {minCandle['change'].values[0]:.1%}, {YMDBetweenDates(minCandle.iloc[0]['timestamp'], bearData[i].iloc[0]['timestamp'])}since peak ({minCandle.iloc[0]['days']} trading days)''',
		align='left',
		showarrow=False,
		yshift=-8,
		xshift=-6,
		xanchor='left'
		)

	# Cycle end
	fig.add_annotation(x=bearData[i].iloc[-1]['days'], 
		y=bearData[i].iloc[-1]['change'],
		text=f'''{YMDBetweenDates(bearData[i].iloc[-1]['timestamp'], bearData[i].iloc[0]['timestamp'])}since peak ({bearData[i].iloc[-1]['days']} trading days)   <br>
		{YMDBetweenDates(bearData[i].iloc[-1]['timestamp'], minCandle.iloc[0]['timestamp'])}since market bottom ({bearData[i].iloc[-1]['days'] - minCandle.iloc[0]['days']} trading days) ┐''',
		align='right',
		showarrow=False,
		yshift=17,
		xshift=7,
		xanchor='right',
		ax=0, ay=0
		)

	fig.update_layout( 
		yaxis=dict(tickformat="0.2%"),
		xaxis_title = "Trading days since last peak 🠆",
		yaxis_title = "🠄 Drawdown",
		template='simple_white',
		paper_bgcolor="rgb(255,255,255)", plot_bgcolor="rgb(255,255,255)",
		legend_traceorder="reversed",
		hovermode='x unified',
		margin=dict(t=50),
		width=1200,
		title=f'Comparing {name} With Present Day'
	)
	fig.show()

In [37]:
layoffs_2008 = pd.read_csv('2008-layoffs-forbes.csv', encoding='ISO-8859-1')

layoffs_2008['Latest Layoff'] = pd.to_datetime(layoffs_2008['Latest Layoff'], infer_datetime_format=True)

layoffs_2008_filtered = layoffs_2008[layoffs_2008['Total Laid Off to Date'] > 100 & (layoffs_2008['Latest Layoff'] > bearData[9]['timestamp'].min()) & (layoffs_2008['Latest Layoff'] < bearData[9]['timestamp'].max())]

In [43]:
[x for x in layoffs_2008_filtered.iterrows()]

[(0,
  Latest Layoff             2010-03-31 00:00:00
  Company                                   ITT
  Total Laid Off to Date                     45
  Industry                         Conglomerate
  Name: 0, dtype: object),
 (1,
  Latest Layoff             2010-03-30 00:00:00
  Company                                 Xerox
  Total Laid Off to Date                   2774
  Industry                    Business Services
  Name: 1, dtype: object),
 (2,
  Latest Layoff             2010-03-29 00:00:00
  Company                              Allstate
  Total Laid Off to Date                    124
  Industry                            Insurance
  Name: 2, dtype: object),
 (3,
  Latest Layoff              2010-03-24 00:00:00
  Company                             JM Smucker
  Total Laid Off to Date                     700
  Industry                  Food Drink & Tobacco
  Name: 3, dtype: object),
 (4,
  Latest Layoff             2010-03-17 00:00:00
  Company                                  AT&T

In [35]:
bearData[9]

,timestamp,open,volume,close,high,low,adjclose,days,change
20030,2007-10-09 13:30:00,1553.180054,2932040000,1565.150024,1565.260010,1551.819946,1565.150024,0,NaN
20031,2007-10-10 13:30:00,1564.979980,3044760000,1562.469971,1565.420044,1555.459961,1562.469971,1,-0.001712
20032,2007-10-11 13:30:00,1564.719971,3911260000,1554.410034,1576.089966,1546.719971,1554.410034,2,-0.006862
20033,2007-10-12 13:30:00,1555.410034,2788690000,1561.800049,1563.030029,1554.089966,1561.800049,3,-0.002140
20034,2007-10-15 13:30:00,1562.250000,3139290000,1548.709961,1564.739990,1540.810059,1548.709961,4,-0.010504
...,...,...,...,...,...,...,...,...,...
21402,2013-03-22 13:30:00,1545.900024,2948380000,1556.890015,1557.739990,1545.900024,1556.890015,1372,-0.005277
21403,2013-03-25 13:30:00,1556.890015,3178170000,1551.689941,1564.910034,1546.219971,1551.689941,1373,-0.008600
21404,2013-03-26 13:30:00,1551.689941,2869260000,1563.770020,1563.949951,1551.689941,1563.770020,1374,-0.000882
21405,2013-03-27 13:30:00,1563.750000,2914210000,1562.849976,1564.069946,1551.900024,1562.849976,1375,-0.001470


In [39]:
fig = go.Figure(data=[
    go.Bar(name='SF Zoo', x=layoffs_2008_filtered['Latest Layoff'], y=layoffs_2008_filtered['Total Laid Off to Date']),
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [33]:
for i, data in reversed(list(enumerate(bearData[:-1]))):
	name = data.iloc[0]['timestamp'].strftime("%b-%Y")
	if name != 'Oct-2007':
		continue
	fig = go.Figure()
	fig.add_trace(go.Scatter(x=currentMarket['days'], y=currentMarket['change'], mode='lines', name='Current Market', line={'width': 3, 'color': 'darkblue'}, 
		hovertemplate=[f" {x['change']:.1%} ({x['timestamp'].strftime('%b %d, %Y')})" for i, x in currentMarket.iterrows()]
		))
	fig.add_trace(go.Scatter(x=bearData[i]['days'], y=bearData[i]['change'], mode='lines', name=name,
		hovertemplate=[f" {x['change']:.1%} ({x['timestamp'].strftime('%b %d, %Y')})" for i, x in bearData[i].iterrows()]
		))

	minCandle = bearData[i][bearData[i].close == bearData[i].close.min()]

	# Peak
	fig.add_annotation(x=0, 
		y=0,
		text=f'''┌ Peak''',
		align='left',
		showarrow=False,
		yshift=9,
		xshift=-5,
		xanchor='left'
		)

	# Market bottom
	fig.add_annotation(x=minCandle['days'].values[0], 
		y=minCandle['change'].values[0],
		text=f'''└ {minCandle['change'].values[0]:.1%}, {YMDBetweenDates(minCandle.iloc[0]['timestamp'], bearData[i].iloc[0]['timestamp'])}since peak ({minCandle.iloc[0]['days']} trading days)''',
		align='left',
		showarrow=False,
		yshift=-8,
		xshift=-6,
		xanchor='left'
		)

	# Cycle end
	fig.add_annotation(x=bearData[i].iloc[-1]['days'], 
		y=bearData[i].iloc[-1]['change'],
		text=f'''{YMDBetweenDates(bearData[i].iloc[-1]['timestamp'], bearData[i].iloc[0]['timestamp'])}since peak ({bearData[i].iloc[-1]['days']} trading days)   <br>
		{YMDBetweenDates(bearData[i].iloc[-1]['timestamp'], minCandle.iloc[0]['timestamp'])}since market bottom ({bearData[i].iloc[-1]['days'] - minCandle.iloc[0]['days']} trading days) ┐''',
		align='right',
		showarrow=False,
		yshift=17,
		xshift=7,
		xanchor='right',
		ax=0, ay=0
		)

	fig.update_layout( 
		yaxis=dict(tickformat="0.2%"),
		xaxis_title = "Trading days since last peak 🠆",
		yaxis_title = "🠄 Drawdown",
		template='simple_white',
		paper_bgcolor="rgb(255,255,255)", plot_bgcolor="rgb(255,255,255)",
		legend_traceorder="reversed",
		hovermode='x unified',
		margin=dict(t=50),
		width=1200
	)
	fig.show()